Original Paper: [U2-Net: Going Deeper with Nested U-Structure for Salient Object Detection](https://arxiv.org/pdf/2005.09007.pdf)

Source: https://github.com/danielgatis/rembg

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from google.colab.patches import cv2_imshow

In [ ]:
!pip install rembg[gpu]

In [ ]:
from rembg import remove

In [ ]:
# Processing the image
input = cv2.imread('/content/.../.../....JPG')
input = cv2.cvtColor(input, cv2.COLOR_BGR2RGB)

# Removing the background from the given Image
output_img = remove(input)

In [ ]:
plt.imshow(output_img[:,:,:3])

In [ ]:
# Load the segmented image
#image = cv2.imread('/content/output/img.png', cv2.IMREAD_GRAYSCALE)
output_img_gray = tf.image.rgb_to_grayscale(output_img[:,:,:3])
output_img_gray = np.squeeze(output_img_gray)

# Find the coordinates of the non black pixels
nonzero_pixels = np.argwhere(output_img_gray > 0)

In [ ]:
# Get the bounding box for the image
y_min, x_min = np.min(nonzero_pixels, axis=0)
y_max, x_max = np.max(nonzero_pixels, axis=0)

print(f"(x1, y1): ({x_min}, {y_min})")
print(f"(x2, y2): ({x_max}, {y_max})")

(x1, y1): (603, 800)
(x2, y2): (3708, 3461)


In [ ]:
# Crop the masked region from the original image
cropped_img = output_img[y_min:y_max+1, x_min:x_max+1]

# Save or display the cropped PCB image
cv2.imwrite('1.png', cropped_img)
cv2_imshow(cropped_img)

In [ ]:
def remove_background_img(img_dir, show=False, resize_shape=None, save=True, save_name="img"):
  # Processing the image
  input = cv2.imread(img_dir)
  input = cv2.cvtColor(input, cv2.COLOR_BGR2RGB)

  # Removing the background from the given Image
  output_img = remove(input)

  # Load the segmented image
  output_img_gray = tf.image.rgb_to_grayscale(output_img[:,:,:3])
  output_img_gray = np.squeeze(output_img_gray)

  # Find the coordinates of the non black pixels
  nonzero_pixels = np.argwhere(output_img_gray > 0)

  # Get the bounding box for the image
  y_min, x_min = np.min(nonzero_pixels, axis=0)
  y_max, x_max = np.max(nonzero_pixels, axis=0)

  # Crop the masked region from the original image
  cropped_img = output_img[y_min:y_max+1, x_min:x_max+1]
  cropped_img = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB)

  if resize_shape != None:
    cropped_img = (tf.image.resize(cropped_img, resize_shape)).numpy()

  # Save or display the cropped PCB image
  if save:
    save_name = str(save_name) + ".png"
    cv2.imwrite(save_name, cropped_img)
  if show:
    cv2_imshow(cropped_img)

In [ ]:
# remove_background_img('/content/../../../img.JPG', show=True, save_name='img_cropped')

In [ ]:
import glob

def get_image_file_list(dir, ext):
    formats = ['jpg', 'png', 'jpeg', 'JPG', 'JPEG', 'PNG']
    img_list = []

    if ext in formats:
        rgx_for_img = dir + "/*." + ext
        img_list = glob.glob(rgx_for_img)

    return img_list

In [ ]:
img_files = get_image_file_list(IMG_DIR, 'JPG')

In [ ]:
print("Number of flat image files: ", len(img_files))

In [ ]:
for ix in range(len(img_files)):
  save_path = '/content/.../.../cropped/'
  save_name_str = save_path + str(img_files[ix][-12:-4]) + '_cropped'
  remove_background_img(img_files[ix], show=False, resize_shape=(1344, 1792), save_name=save_name_str)